Data Cleaning

In [29]:
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt



In [30]:
#apple_df = pd.read_csv('https://media.githubusercontent.com/media/aquonAttheHub/207_datasets/main/apple_processed.csv')

In [31]:
apple = "apple"
tesla = "tesla"
apple_df = pd.read_csv(apple + '_stock_news.zip')
tesla_df = pd.read_csv(tesla + '_stock_news.zip')

Apple Stock Price Cleaning

In [32]:
apple_df.describe()

,Open,High,Low,Close,Volume
count,865671.000000,865671.000000,865671.000000,865671.000000,8.656710e+05
mean,80.321309,80.372577,80.259182,80.319125,2.502713e+05
std,55.150437,55.176618,55.127715,55.151726,3.499110e+05
min,20.634000,20.670000,20.604000,20.635000,2.100000e+01
25%,32.894000,32.915000,32.857000,32.890000,9.912900e+04
50%,50.863000,50.898000,50.821000,50.862000,1.666040e+05
75%,136.595000,136.697000,136.493000,136.595000,2.928120e+05
max,197.504000,197.964000,196.962000,197.503000,4.261870e+07


In [33]:
#Check for Missing Values
pd.isna(apple_df).value_counts()

Date   Open   High   Low    Close  Volume  News 
False  False  False  False  False  False   False    865671
Name: count, dtype: int64

In [34]:
tesla_df.head()

,Date,Open,High,Low,Close,Volume,News
0,2015-01-02 09:30:00,14.858,14.883,14.810,14.865,428550,
1,2015-01-02 09:31:00,14.862,14.876,14.813,14.845,152475,
2,2015-01-02 09:32:00,14.845,14.867,14.832,14.834,165600,
3,2015-01-02 09:33:00,14.840,14.847,14.820,14.820,60000,
4,2015-01-02 09:34:00,14.820,14.847,14.810,14.811,415800,


In [35]:
#It will be useful to convert the Date strings to Datetime objects.
apple_df["Date"] = pd.to_datetime(apple_df["Date"])
apple_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 865671 entries, 0 to 865670
Data columns (total 7 columns):
 #   Column  Non-Null Count   Dtype         
---  ------  --------------   -----         
 0   Date    865671 non-null  datetime64[ns]
 1   Open    865671 non-null  float64       
 2   High    865671 non-null  float64       
 3   Low     865671 non-null  float64       
 4   Close   865671 non-null  float64       
 5   Volume  865671 non-null  int64         
 6   News    865671 non-null  object        
dtypes: datetime64[ns](1), float64(4), int64(1), object(1)
memory usage: 46.2+ MB


In [36]:
apple_df["Date"].head()

0   2015-01-02 09:30:00
1   2015-01-02 09:31:00
2   2015-01-02 09:32:00
3   2015-01-02 09:33:00
4   2015-01-02 09:34:00
Name: Date, dtype: datetime64[ns]

## Transform minutely data to daily data.

In [37]:
def preprocess(df: pd.DataFrame) -> pd.DataFrame:
    df["Date"] = pd.to_datetime(df['Date'])
    df["Date"] = df["Date"].apply(lambda x: x.date())
    last_close = df.groupby("Date")['Close'].last()
    first_open = df.groupby("Date")['Close'].first()
    lowest = df.groupby("Date")['Low'].min()
    highest = df.groupby("Date")['High'].max()
    
    total_vol = df.groupby("Date")['Volume'].sum()
    mean_vol = df.groupby("Date")['Volume'].mean()
    std_vol = df.groupby("Date")['Volume'].std()
    news = df.groupby("Date")['News'].last()
    result = pd.DataFrame({
        "close": last_close,
        "open": first_open,
        "lowest": lowest,
        "highest": highest,
        "total_vol": total_vol,
        "mean_vol": mean_vol,
        "std_vol": std_vol,
        "news": news,
    })
    result.reset_index(inplace = True)
    result["Date"] = pd.to_datetime(result['Date'])
    result["month"] = result["Date"].dt.month.astype('category')
    result["day"] = result["Date"].dt.day.astype('category')
    result["is_up"] = (result["close"].diff(-1) < 0).astype(int)
    
    
    return result


In [38]:
data_apple = preprocess(apple_df)
data_tesla = preprocess(tesla_df)

In [39]:
data_apple.to_csv("./apple_processed_data.csv")
data_tesla.to_csv("./tesla_processed_data.csv")

In [40]:
data_apple.head()

,Date,close,open,lowest,highest,total_vol,mean_vol,std_vol,news,month,day,is_up
0,2015-01-02,24.490,24.925,24.026,24.992,188181988,482517.917949,453958.983288,"[""Is Xiaomi's first laptop a MacBook Air knock...",1,2,0
1,2015-01-05,23.807,24.192,23.592,24.366,200586492,514324.338462,426711.446836,"[""Is Xiaomi's first laptop a MacBook Air knock...",1,5,0
2,2015-01-06,23.802,23.842,23.417,24.092,237766160,609656.820513,452106.984355,"[""Is Xiaomi's first laptop a MacBook Air knock...",1,6,1
3,2015-01-07,24.143,24.017,23.879,24.265,137809632,353358.030769,315345.332594,Apple aims at bendable devices\nA newly issued...,1,7,1
4,2015-01-08,25.066,24.391,24.328,25.151,201020076,515436.092308,344928.788130,"Xiaomi's buying spree gives Apple, Samsung mor...",1,8,1


In [41]:
temp = data_apple['is_up'].value_counts()
temp

is_up
1    1170
0    1053
Name: count, dtype: int64

In [42]:
baseline = temp[1] / np.sum(temp)
print(baseline * 100)

52.63157894736842


In [43]:
temp = data_tesla['is_up'].value_counts()
baseline = temp[1] / np.sum(temp)
print(baseline * 100)

51.956815114709855
